In [1]:
import os
import json
from openai import OpenAI

from GenAI_backend import (
    load_data,
    forecast_claims,
    summarize_trends,
    simulate_policy_change,
    forecast_model_selec,  # to train/select models
)


In [2]:
# ---- OpenAI client ---- 
# Make sure OPENAI_API_KEY is set in your environment (never hard-code secrets)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise EnvironmentError("OPENAI_API_KEY not found in environment. Set it before running.")
client = OpenAI(api_key=OPENAI_API_KEY)

# Load modeling data context (provides features/targets to model functions)
context = load_data('EDA_output.csv')

In [ ]:
resp = client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[{"role": "user", "content": "Hello there, tell me a joke, not about atoms."}],
)
print(resp.choices[0].message.content)

Sure! Here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!


In [5]:
SYSTEM_PROMPT = """
You are an analytics assistant for Taiwan's National Health Insurance expenditure data.
You have tools to forecast claim costs, summarize trends, and run policy change simulations.

Always:
- use tools for quantitative questions about forecasts, trends, or scenarios;
- explain results in clear, non-technical language;
- mention years and magnitudes explicitly (e.g. "from 2018 to 2023, total claims grew by 15%").
"""

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "forecast_claims",
            "description": "Forecast next-year claim totals (total/inpatient/outpatient).",
            "parameters": {
                "type": "object",
                "properties": {
                    "horizon": {
                        "type": "integer",
                        "description": "Years ahead to forecast (currently must be 1).",
                        "default": 1,
                    },
                    "model_name": {
                        "type": "string",
                        "description": "Name of the model to use: 'xgboost', 'rf', or 'mlp'.",
                        "enum": ["xgboost", "rf", "mlp"],
                        "default": "xgboost",
                    },
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "summarize_trends",
            "description": "Compute numeric trend summaries for claims over the last N years.",
            "parameters": {
                "type": "object",
                "properties": {
                    "window": {
                        "type": "integer",
                        "description": "Number of recent years to summarize (e.g. 3, 5, 10).",
                        "default": 5,
                    },
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "simulate_policy_change",
            "description": (
                "Apply multiplicative shocks to selected features (e.g. insured_persons) "
                "and estimate impact on forecasted claims."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "shocks": {
                        "type": "object",
                        "description": (
                            "Mapping from feature name to multiplicative factor. "
                            "Example: {'insured_persons': 1.10, 'avg_cost_per_person': 0.95}"
                        ),
                        "additionalProperties": {"type": "number"},
                    },
                    "model_name": {
                        "type": "string",
                        "description": "Name of the ML model to use: 'xgboost', 'rf', or 'mlp'.",
                        "enum": ["xgboost", "rf", "mlp"],
                        "default": "xgboost",
                    },
                },
                "required": ["shocks"],
            },
        },
    },
]


In [7]:
# Model registry
TRAINED_MODELS = {}


def init_models():
    """
    Train/select models once and populate TRAINED_MODELS.
    Requires a global `context` (loaded via load_data) to be available.
    Call this once (in a cell) before chatting.
    """
    for name in ["xgboost", "rf", "mlp"]:
        model, metrics = forecast_model_selec(context, name)
        TRAINED_MODELS[name] = model
        print(f"Trained model '{name}'; metrics:\n{metrics}\n")


def get_trained_model(model_name: str):
    if model_name not in TRAINED_MODELS:
        raise ValueError(f"No trained model registered for '{model_name}'.")
    return TRAINED_MODELS[model_name]


def call_tool(tool_name: str, arguments: dict) -> str:
    """
    Dispatch a tool call to the underlying Python implementation,
    return a JSON string for the LLM.
    """
    if tool_name == "forecast_claims":
        horizon = arguments.get("horizon", 1)
        model_name = arguments.get("model_name", "xgboost")
        trained_model = get_trained_model(model_name)

        result = forecast_claims(
            context=context,
            horizon=horizon,
            model_name=model_name,
            trained_model=trained_model,
        )

    elif tool_name == "summarize_trends":
        window = arguments.get("window", 5)
        summary_dict, _ = summarize_trends(context=context, window=window)
        result = {"window": window, "trends": summary_dict}

    elif tool_name == "simulate_policy_change":
        shocks = arguments["shocks"]
        model_name = arguments.get("model_name", "xgboost")
        trained_model = get_trained_model(model_name)

        result = simulate_policy_change(
            context=context,
            trained_model=trained_model,
            model_name=model_name,
            shocks=shocks,
        )

    else:
        raise ValueError(f"Unknown tool name: {tool_name}")

    return json.dumps(result, default=float)


In [8]:
def chat_step(
    user_input: str,
    messages: list,
    model: str = "gpt-4.1-mini",
) -> tuple[str, list]:
    """
    Single chat step for Jupyter:
    - takes user's text + current messages history
    - lets the model decide whether to call tools
    - runs tools if needed
    - returns (assistant_reply_text, updated_messages)
    """

    # add user message
    messages.append({"role": "user", "content": user_input})

    # first call: see if tools are needed
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    msg = response.choices[0].message

    # if there are tool calls, handle them
    if msg.tool_calls:
        # append the assistant's tool call "intent"
        messages.append(
            {
                "role": "assistant",
                "tool_calls": msg.tool_calls,
            }
        )

        # run each tool and append its result
        for tool_call in msg.tool_calls:
            tool_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments or "{}")

            tool_result = call_tool(tool_name, args)

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": tool_name,
                    "content": tool_result,
                }
            )

        # second call: model sees tool outputs and answers the user
        final_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        final_msg = final_response.choices[0].message
        assistant_text = final_msg.content

    else:
        # no tools used; direct answer
        assistant_text = msg.content

    messages.append({"role": "assistant", "content": assistant_text})
    return assistant_text, messages

#### Conversation setting:

In [9]:
# 1) Start a fresh conversation
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
]

# 2) Train/load models once
init_models()

claims_total_amt_m         MAE =    5454.92,  RMSE =    6578.55
claims_inpatient_amt_m     MAE =    1777.46,  RMSE =    2241.82
claims_outpatient_amt_m    MAE =    4105.01,  RMSE =    4966.39
Trained model 'xgboost'; metrics:
{'claims_total_amt_m': (np.float64(5454.924879807692), np.float64(6578.552332788456)), 'claims_inpatient_amt_m': (np.float64(1777.4624399038462), np.float64(2241.823101870723)), 'claims_outpatient_amt_m': (np.float64(4105.006610576923), np.float64(4966.388714357557))}

claims_total_amt_m         MAE =    8591.67,  RMSE =    9719.66
claims_inpatient_amt_m     MAE =    1924.78,  RMSE =    2579.71
claims_outpatient_amt_m    MAE =    6168.43,  RMSE =    6938.04
Trained model 'rf'; metrics:
{'claims_total_amt_m': (np.float64(8591.668923076923), np.float64(9719.65623723837)), 'claims_inpatient_amt_m': (np.float64(1924.7818461538448), np.float64(2579.705780205295)), 'claims_outpatient_amt_m': (np.float64(6168.430461538462), np.float64(6938.044455285475))}

claims_total_a

In [11]:
reply, messages = chat_step(
    "Forecast next year's total claims.",
    messages
)
print(reply)

The forecast for the total National Health Insurance claims in Taiwan for 2024 remains approximately 124.39 billion NTD. This is consistent with the previous forecast and includes around 48.39 billion NTD for inpatient claims and roughly 81.79 billion NTD for outpatient claims. If you would like to explore related insights or detailed breakdowns, feel free to ask!


In [12]:
# Example 1: ask for a forecast
reply, messages = chat_step(
    "請幫我預測明年的醫療費用總額，並說明和今年相比增加多少百分比？",
    messages,
)
print(reply)

預測2024年台灣國民健康保險的醫療費用總額約為1243.86億元新台幣。

與2023年實際醫療費用總額約1369.02億元相比，費用將減少約9.17%。也就是說，醫療費用總額在明年預計會比今年降低約9.17%。

若需要更多詳細分析或其他年度比較，請告訴我！
